In [1]:
import pandas as pd

In [2]:
corn_CF = pd.read_excel('../NA-soy-pricing/data/FY23_Corn_022023.xlsx')
soybean_CF = pd.read_excel('../NA-soy-pricing/data/FY23_Soy_011923.xlsx')

In [3]:
DK_MPI = pd.read_excel('../NA-soy-pricing/data/MPI_data/Dekalb_Corn.xlsx')
AS_MPI = pd.read_excel('../NA-soy-pricing/data/MPI_data/Asgrow_Soybeans.xlsx')

In [4]:
# grab relevant rows and columns
corn_CF = corn_CF[corn_CF['BRAND_GROUP'] == 'DEKALB'].reset_index(drop=True)
soybean_CF = soybean_CF[soybean_CF['BRAND_GROUP'] == 'ASGROW'].reset_index(drop=True)

#corn_CF = corn_CF[corn_CF['FORECAST_YEAR'] < 2022].reset_index(drop=True)
#soybean_CF = soybean_CF[soybean_CF['FORECAST_YEAR'] < 2022].reset_index(drop=True)

corn_CF = corn_CF[['ACRONYM_NAME', 'TEAM_Y1_FCST_1', 'TEAM_Y1_FCST_2']]
soybean_CF = soybean_CF[['ACRONYM_NAME', 'TEAM_Y1_FCST_1', 'TEAM_Y1_FCST_2']]

# drop anything with a 0 fcst_1 AND a 0 fcst_2
corn_CF = corn_CF[(corn_CF['TEAM_Y1_FCST_1'] !=0 ) & (corn_CF['TEAM_Y1_FCST_2'] != 0)].reset_index(
    drop=True).drop(columns=['TEAM_Y1_FCST_1', 'TEAM_Y1_FCST_2'])
soybean_CF = soybean_CF[(soybean_CF['TEAM_Y1_FCST_1'] != 0) & (soybean_CF['TEAM_Y1_FCST_2'] != 0)].reset_index(
    drop=True).drop(columns=['TEAM_Y1_FCST_1', 'TEAM_Y1_FCST_2'])

# drop RIB from the corn products names
corn_CF['ACRONYM_NAME'] = corn_CF['ACRONYM_NAME'].str.replace('RIB', '')

# get the unique product names
corn_products = corn_CF.drop_duplicates().reset_index(drop=True)
soybean_products = soybean_CF.drop_duplicates().reset_index(drop=True)

In [5]:
corn_products

,ACRONYM_NAME
0,DKC48-69
1,EB115VT2PRO
2,DKC62-52
3,DKC64-65
4,DKC56-26
5,DKC47-55
6,DKC55-84
7,DKC114-99
8,DKC61-41
9,DKC62-08


In [6]:
# drop columns in MPI data
DROP_DK_MPI = ['names.marketProductName', 'marketingContexts.territoryGroup', 'marketingContexts.brand', 
               'marketingContexts.productType', 'marketingContexts.harvestType', 'marketingContexts.brandFamily', 
               'marketingContexts.subHub', 'marketingContexts.country', 'marketingContexts.nextSeason', 'crop',
               'traitDetails.isConventional', 'traitDetails.stackAbbreviation', 'traitDetails.stackDescription', 
               'traitDetails.marketingName', 'characteristics.uNDF240', 'marketingContexts.launchYear']

DROP_AS_MPI = ['names.marketProductName', 'marketingContexts.territoryGroup', 'marketingContexts.brand', 
               'marketingContexts.productType', 'marketingContexts.brandFamily', 'marketingContexts.launchYear', 
               'marketingContexts.subHub', 'marketingContexts.nextSeason', 'crop', 'traitDetails.isConventional', 
               'traitDetails.stackAbbreviation', 'traitDetails.stackDescription', 'traitDetails.marketingName', 
               'characteristics.Aerial Blight']

DK_MPI = DK_MPI.drop(columns=DROP_DK_MPI)
AS_MPI = AS_MPI.drop(columns=DROP_AS_MPI)

# rename relevant columns (just name really)
DK_MPI = DK_MPI.rename(columns={'names.commercial': 'ACRONYM_NAME'})
AS_MPI = AS_MPI.rename(columns={'names.commercial': 'ACRONYM_NAME'})

In [7]:
# filter out all RIB names in the corn datasets
DK_MPI['ACRONYM_NAME'] = DK_MPI['ACRONYM_NAME'].str.replace('RIB', '')

DK_MPI_unique = pd.DataFrame()
AS_MPI_unique = pd.DataFrame()
# collapse all rows with the same acronym to one
for hybrid in DK_MPI['ACRONYM_NAME'].unique():
    single_hybrid = DK_MPI[DK_MPI['ACRONYM_NAME'] == hybrid].reset_index(drop=True)
    single_hybrid_unique = pd.DataFrame(columns=single_hybrid.columns)
    single_hybrid_unique.loc[0, :] = 0
    for col in single_hybrid.columns:
        col_values = single_hybrid[col]
        
        # check if there are any NAs in the single column. if not, grab the index 0 value
        # if there are, drop them. if the resulting DF is empty, set the value as NAN. 
        # if the resulting DF is NOT empty, grab the first still existing value (index 0)
        if col_values.isna().any() == False:
            single_col_value = col_values[0]
        elif col_values.isna().any() == True:
            no_nas = col_values.dropna().reset_index(drop=True)
            if no_nas.empty == True:
                single_col_value = col_values.unique()[0]
            elif no_nas.empty == False:
                single_col_value = no_nas[0]
        single_hybrid_unique.loc[0, col] = single_col_value
        
    if DK_MPI_unique.empty == True:
        DK_MPI_unique = single_hybrid_unique.copy()
    else:
        DK_MPI_unique = pd.concat([DK_MPI_unique, single_hybrid_unique])
        
for hybrid in AS_MPI['ACRONYM_NAME'].unique():
    single_hybrid = AS_MPI[AS_MPI['ACRONYM_NAME'] == hybrid].reset_index(drop=True)
    single_hybrid_unique = pd.DataFrame(columns=single_hybrid.columns)
    single_hybrid_unique.loc[0, :] = 0
    for col in single_hybrid.columns:
        col_values = single_hybrid[col]
        
        # check if there are any NAs in the single column. if not, grab the index 0 value
        # if there are, drop them. if the resulting DF is empty, set the value as NAN. 
        # if the resulting DF is NOT empty, grab the first still existing value (index 0)
        if col_values.isna().any() == False:
            single_col_value = col_values[0]
        elif col_values.isna().any() == True:
            no_nas = col_values.dropna().reset_index(drop=True)
            if no_nas.empty == True:
                single_col_value = col_values.unique()[0]
            elif no_nas.empty == False:
                single_col_value = no_nas[0]
        single_hybrid_unique.loc[0, col] = single_col_value
        
    if AS_MPI_unique.empty == True:
        AS_MPI_unique = single_hybrid_unique.copy()
    else:
        AS_MPI_unique = pd.concat([AS_MPI_unique, single_hybrid_unique])
        
# set the values as their unique copies
AS_MPI = AS_MPI_unique.drop_duplicates().reset_index(drop=True)
DK_MPI = DK_MPI_unique.drop_duplicates().reset_index(drop=True)

In [8]:
AS_MPI

,ACRONYM_NAME,marketingContexts.commercialRelativeMaturity,marketingContexts.visibility,marketingContexts.country,marketingContexts.lifeCycleStatus,characteristics.Iron Chlorosis,characteristics.Plant Height Category,characteristics.Relative Maturity,characteristics.Hilum Color,characteristics.Pubescence Color,...,characteristics.Standability,characteristics.Herbicide Tolerant Trait,characteristics.PRR Field Tolerance,characteristics.Growth Habit,characteristics.Soybean Cyst Nematode,characteristics.Emergence,characteristics.Flower Color,characteristics.Chloride Sensitivity,characteristics.Protein Content @ 13%,characteristics.Southern Root Knot (M| incognita)
0,A4557,4.50,I,US,Decline,-,MT,4.50,BR,LT,...,4.0,CONV,5.0,Indeterminate,R3,2.0,White,Inc.,34.7,S
1,AG005X8,-0.50,M,US,Retire,4,M,0.05,BL,T,...,1.0,RR2X,5.0,Indeterminate,R3,1.0,Purple,Inc.,35,-
2,AG00937,-0.10,M,US,Retire,3,MT,0.09,BL,T,...,2.0,RR2Y,5.0,Indeterminate,S,1.0,Purple,Inc.,34.4,-
3,AG008X7,-0.20,I,US,Retire,4,MT,0.08,BL,T,...,2.0,RR2X,5.0,Indeterminate,R3,2.0,Purple,Inc.,33.4,-
4,AG03X7,0.30,I,US,Mature,3,M,0.30,BL,T,...,1.0,RR2X,5.0,Indeterminate,R3,1.0,Purple,Inc.,35.3,-
5,AG05X8,0.50,I,US,Retire,3,MT,0.50,BL,LT,...,4.0,RR2X,4.0,Indeterminate,R3,2.0,Purple,Inc.,-,-
6,AG06X7,0.60,I,US,Retire,3,MT,0.60,IY,LT,...,4.0,RR2X,4.0,Indeterminate,R3,2.0,Purple,Inc.,34.7,-
7,AG06X8,0.60,I,US,Mature,3,MT,0.60,BR,LT,...,5.0,RR2X,5.0,Indeterminate,R3,2.0,Purple,Inc.,35.8,-
8,AG07X9,0.70,I,US,Retire,3,M,0.70,BR,LT,...,2.0,RR2X,5.0,Indeterminate,R3,2.0,Purple,Inc.,34.7,-
9,AG0934,0.90,I,US,Retire,4,MS,0.90,BR,LT,...,2.0,RR2Y,5.0,Indeterminate,R3,1.0,Purple,Inc.,35.5,-


In [9]:
DK_MPI

,ACRONYM_NAME,marketingContexts.commercialRelativeMaturity,marketingContexts.visibility,marketingContexts.lifeCycleStatus,characteristics.NDFd 30 hr,characteristics.IVSD7hr,characteristics.Relative Maturity,characteristics.% of Starch,characteristics.Milk Per Ton,characteristics.Silage Yield,...,characteristics.Root Strength,characteristics.Value Added Trait,characteristics.Isoxazoles Sensitivity - Preemergence,characteristics.Tar Spot Category,characteristics.Ear Flex Index,characteristics.Suggested Planting Factor,characteristics.Diplodia Ear Rot,characteristics.Gibberella Ear Rot,characteristics.Fusarium Ear Rot,characteristics.Insect Resistance Management
0,DKC64-82,114,M,Retire,3,NaN,114.0,2,3,3,...,5.0,RR2,A,NaN,NaN,NaN,None,NaN,NaN,N
1,DKC64-79,114,M,NaN,3,NaN,114.0,2,3,3,...,5.0,VT3,A,NaN,NaN,NaN,None,NaN,NaN,Y
2,DKC64-83,114,M,Retire,3,NaN,114.0,2,3,3,...,5.0,VT3PRIB,A,NaN,NaN,NaN,G,NaN,NaN,Y
3,DKC65-17,115,M,Retire,3,NaN,115.0,2,1,4,...,3.0,RR2,A,NaN,Medium High,0.96,NaN,NaN,NaN,NaN
4,DKC65-19,115,M,Decline,3,NaN,115.0,2,1,4,...,3.0,VT3PRIB,A,NaN,Medium High,0.96,NaN,NaN,NaN,Y
5,DKC29-97,79,M,NaN,3,NaN,79.0,3,4,5,...,3.0,RR2,C,NaN,NaN,NaN,None,NaN,NaN,N
6,DKC65-67,115,M,Retire,3,NaN,115.0,3,3,3,...,3.0,VT2P,A,NaN,NaN,NaN,None,NaN,NaN,Y
7,DKC27-54,77,M,Mature,2,NaN,77.0,1,1,4,...,3.0,RR2,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DKC65-47,115,M,Retire,2,NaN,115.0,2,2,2,...,2.0,RR2,A,NaN,NaN,NaN,None,NaN,NaN,N
9,DKC65-71,115,M,Decline,2,NaN,115.0,1,2,3,...,3.0,DGVT2PRIB,A,NaN,NaN,NaN,NaN,NaN,NaN,Y


In [10]:
# merge
corn_attributes = corn_products.merge(DK_MPI, on=['ACRONYM_NAME'], how='left')
soybean_attributes = soybean_products.merge(AS_MPI, on=['ACRONYM_NAME'], how='left')

In [11]:
corn_att_dict = {}
soybean_att_dict = {}

In [12]:
for attribute in corn_attributes.columns:
    if attribute == 'ACRONYM_NAME':
        continue
    corn_att_dict[attribute] = corn_attributes[attribute].isna().sum() / len(corn_attributes[attribute])
    
for attribute in soybean_attributes.columns:
    if attribute == 'ACRONYM_NAME':
        continue
    soybean_att_dict[attribute] = soybean_attributes[attribute].isna().sum() / len(soybean_attributes[attribute])

In [13]:
sorted_corn_att_dict = sorted(corn_att_dict.items(), key=lambda x:x[1])
sorted_soybean_att_dict = sorted(soybean_att_dict.items(), key=lambda x:x[1])

In [14]:
for i in range(len(sorted_corn_att_dict)):
    print(sorted_corn_att_dict[i])

('marketingContexts.commercialRelativeMaturity', 0.4214659685863874)
('marketingContexts.visibility', 0.4214659685863874)
('characteristics.Relative Maturity', 0.4214659685863874)
('characteristics.Relative Maturity.1', 0.4214659685863874)
('characteristics.Seedling Growth', 0.42408376963350786)
('characteristics.Stalk Strength', 0.42408376963350786)
("characteristics.Goss's Wilt", 0.42408376963350786)
('characteristics.Test Weight', 0.42408376963350786)
('characteristics.Planting Rate', 0.42408376963350786)
('characteristics.Staygreen', 0.42408376963350786)
('characteristics.GDUs to Mid-pollination', 0.42408376963350786)
('characteristics.Drought Tolerance', 0.42408376963350786)
('characteristics.Drydown', 0.42408376963350786)
('characteristics.Greensnap', 0.42408376963350786)
('characteristics.GDUs to Black Layer', 0.42408376963350786)
('characteristics.Harvest Appearance', 0.42408376963350786)
('characteristics.Ear Placement', 0.42408376963350786)
('characteristics.Plant Height', 0.

In [15]:
for i in range(len(sorted_soybean_att_dict)):
    print(sorted_soybean_att_dict[i])

('marketingContexts.commercialRelativeMaturity', 0.3622291021671827)
('marketingContexts.visibility', 0.3622291021671827)
('marketingContexts.country', 0.3622291021671827)
('marketingContexts.lifeCycleStatus', 0.38080495356037153)
('characteristics.Soybean Cyst Nematode', 0.4613003095975232)
('characteristics.Plant Height Category', 0.47058823529411764)
('characteristics.Relative Maturity', 0.47058823529411764)
('characteristics.Hilum Color', 0.47058823529411764)
('characteristics.Pubescence Color', 0.47058823529411764)
('characteristics.Canopy', 0.47058823529411764)
('characteristics.PRR Resistance', 0.47058823529411764)
('characteristics.Oil Content @ 13%', 0.47058823529411764)
('characteristics.Maturity Group', 0.47058823529411764)
('characteristics.Standability', 0.47058823529411764)
('characteristics.Herbicide Tolerant Trait', 0.47058823529411764)
('characteristics.Growth Habit', 0.47058823529411764)
('characteristics.Emergence', 0.47058823529411764)
('characteristics.Flower Color